**Escrevendo o arquivo novo**

In [7]:
# instalar dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install pyspark
!pip install -q findspark


In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import findspark
findspark.init()

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [12]:
#COTISTAS
path = "/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/spark/cotistas"
cotistas = spark.read.csv(path, sep=";", inferSchema=True);

#EMPRESAS
path = "/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/spark/empresas"
empresas = spark.read.csv(path, sep=";", inferSchema=True);

#UNIDADES
path = "/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/spark/unidades"
unidades = spark.read.csv(path, sep=";", inferSchema=True);


In [13]:
cotistas.write.csv(
    path='/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/spark/cotistas/csv',
    mode='overwrite',
    sep=';',
    header=True
)

empresas.write.csv(
    path='/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/spark/empresas/csv',
    mode='overwrite',
    sep=';',
    header=True
)

unidades.write.csv(
    path='/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/spark/unidades/csv',
    mode='overwrite',
    sep=';',
    header=True
)

In [15]:
cotistas\
  .select('nome_do_socio_ou_razao_social')\
  .filter(cotistas.nome_do_socio_ou_razao_social.startswith('RODRIGO'))\
  .filter(cotistas.nome_do_socio_ou_razao_social.endswith('DIAS'))\
  .limit(10)\
  .toPandas()

AnalysisException: ignored

In [ ]:
cotistas.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in cotistas.columns]).show()

In [ ]:
impostos = spark.createDataFrame(
    [
        ('Bebidas', 0.15),
        ('Limpeza', 0.05),
        ('Frios', 0.065),
        ('Carnes', 0.08)
    ],
    ['cat', 'tax']
)

In [ ]:
produtos = spark.createDataFrame(
    [
        ('1', 'Bebidas', 'Água mineral'),
        ('2', 'Limpeza', 'Sabão em pó'),
        ('3', 'Frios', 'Queijo'),
        ('4', 'Bebidas', 'Refrigerante'),
        ('5', 'Pet', 'Ração para cães')
    ],
    ['id', 'cat', 'prod']
)

In [ ]:
result = produtos.join(impostos, 'cat', 'inner')
result.show()